# Util Bloqs

The utility bloqs let you reshape Soquets.

In [ ]:
from qualtran.drawing import GraphDrawer, PrettyGraphDrawer
from qualtran.bloqs.util_bloqs import Split, Join, Partition

from IPython.display import SVG

def show_bloq(bloq, draw_cls=PrettyGraphDrawer):
    display(SVG(draw_cls(bloq).get_graph().create_svg()))

## Split and Join

Split and join take a `bitsize=n, shape=()` to `bitsize=1, shape=(n,)` and back.

In [ ]:
show_bloq(Split(3))

In [ ]:
show_bloq(Join(3))

## SplitJoin

For fun, we can pair `Split` and `Join` into an identity operation.

In [ ]:
from attrs import frozen
from functools import cached_property
from typing import *
from qualtran import Bloq, Signature, Register

@frozen
class SplitJoin(Bloq):
    n: int

    @cached_property
    def signature(self) -> Signature:
        return Signature([Register('x', self.n)])

    def build_composite_bloq(
        self, bb: 'BloqBuilder', *, x: 'Soquet'
    ) -> Dict[str, 'Soquet']:
        xs = bb.split(x)
        x = bb.join(xs)
        return {'x': x}

In [ ]:
show_bloq(SplitJoin(10))

In [ ]:
show_bloq(SplitJoin(10).decompose_bloq())

## Partition

A Partition is a useful bloq for abstracting away the details of large registers (like multiple selection registers, spin, orbital, ...). We can use a partition bloq to hide these detailed registers until we decompose the bloq further.

In [ ]:
from qualtran import BloqBuilder, Soquet, SoquetT
@frozen
class ComplicatedBloq(Bloq):
    @cached_property
    def signature(self) -> Signature:
        return Signature(
            [Register('x', 1), Register('y', 4, shape=(3,)), Register('z', 1, shape=(2, 2))]
        )

    def short_name(self) -> str:
        return 'CB'


@frozen
class BlackBoxComplicatedBloq(Bloq):
    @cached_property
    def bitsize(self):
        return sum(reg.total_bits() for reg in self.my_bloq.signature)

    @cached_property
    def my_bloq(self) -> Bloq:
        return ComplicatedBloq()

    def short_name(self) -> str:
        return 'BBCB'

    @cached_property
    def signature(self) -> Signature:
        return Signature.build(regs=self.bitsize)

    def build_composite_bloq(self, bb: 'BloqBuilder', regs: 'SoquetT') -> Dict[str, 'Soquet']:
        bloq_regs = self.my_bloq.signature
        partition = Partition(self.bitsize, bloq_regs)
        out_regs = bb.add(partition, x=regs)
        out_regs = bb.add(self.my_bloq, **{reg.name: sp for reg, sp in zip(bloq_regs, out_regs)})
        regs = bb.add(partition.dagger(), **{reg.name: sp for reg, sp in zip(bloq_regs, out_regs)})
        return {'regs': regs}

In [ ]:
show_bloq(BlackBoxComplicatedBloq())

In [ ]:
show_bloq(BlackBoxComplicatedBloq().decompose_bloq())